In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import scipy.integrate as spi
import pylab as pl
import matplotlib as mlb
from lmfit import minimize, Parameters, Parameter, report_fit
from scipy.signal import savgol_filter
import operator
from pyDOE import *

In [2]:
data2=pd.read_csv("data2.csv")
alive1911=pd.read_csv("alivein1911.csv")
cause=pd.Series(data2['Cause of Death'])
measles=data2[cause.str.contains('measles')==True]
cause=pd.Series(alive1911['Cause of Death'])
measles1911=alive1911[cause.str.contains('measles')==True]

In [3]:
measles_only=np.array([])
for i in range(len(measles1911)):
    
    if measles1911['Cause of Death'].values[i]=='measles':
        measles_only=np.hstack((measles_only,measles1911['Deathdate'].values[i]))

In [4]:
measles_diarrhoea=np.array([])
for i in range(len(measles1911)):
    
    if measles1911['Cause of Death'].values[i]=='measles, diarrhoea':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i]))
    
    elif measles1911['Cause of Death'].values[i]=='measles, diahrroea':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i]))  
    elif measles1911['Cause of Death'].values[i]=='measles, ileo-colitis':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
    elif measles1911['Cause of Death'].values[i]=='measles, diarrhoea,\x0bheart failure':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        
    elif measles1911['Cause of Death'].values[i]=='measles, diarrhoea,\x0bchildbirth':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        
    elif measles1911['Cause of Death'].values[i]== 'measles, diarrhoea,\x0bmiscarriage':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
    
    elif measles1911['Cause of Death'].values[i]== 'measles, gastritis':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        

In [5]:
measles_only=sorted(measles_only,key=lambda x: datetime.strptime(x, '%d/%m/%Y'))
measles_diar=sorted(measles_diarrhoea,key=lambda x: datetime.strptime(x, '%d/%m/%Y'))
deathmeasles=pd.unique(measles_diar)
daterange=pd.date_range(deathmeasles[0],deathmeasles[-1],freq='D')

In [6]:
values3=np.zeros(len(daterange))
deathtime3=np.zeros(len(daterange))
for i in range(len(daterange)):
    for j in range(len(measles_only)):
        if daterange[i]==datetime.strptime(measles_only[j],'%d/%m/%Y'):
            #print(i)
            #print(j)
            values3[i]=values3[i]+1

In [7]:
values2=np.zeros(len(daterange))
deathtime2=np.zeros(len(daterange))
for i in range(len(daterange)):
    for j in range(len(measles_diarrhoea)):
        if daterange[i]==datetime.strptime(measles_diarrhoea[j],'%d/%m/%Y'):
            #print(i)
            #print(j)
            values2[i]=values2[i]+1

In [8]:
measles_timeseries=np.hstack((np.zeros(30),values3,np.zeros(10)))
coinf_timeseries=np.hstack((np.zeros(30),values2,np.zeros(10)))
measles_timeseries2=np.hstack((np.zeros(10),values3,np.zeros(30)))
coinf_timeseries2=np.hstack((np.zeros(10),values2,np.zeros(30)))
measles_timeseries3=np.hstack((np.zeros(1),values3,np.zeros(39)))
coinf_timeseries3=np.hstack((np.zeros(1),values2,np.zeros(39)))

In [9]:
def coinf_sirs_model(INP,t,ps):  
    try:
        beta0=ps['beta0'].value
        beta1=ps['beta1'].value
#     beta=np.hstack((ps['beta0'].value,ps['beta1'].value))
        gamma0=ps['gamma0'].value
        gamma1=ps['gamma1'].value
        c1=ps['c1'].value
        c2=ps['c2'].value
        c3=ps['c3'].value
        c4=ps['c4'].value
        sigma=ps['sigma'].value
        omega=ps['omega'].value
        delta0=ps['delta0'].value

    except:
        beta0,beta1,gamma0,gamma1,c1,c2,c3,c4,sigma,omega,delta0=ps
    N=2618
    
    mu=np.array([c1*omega,c2*omega,c3*omega,c4*gamma1])
    beta=np.hstack((beta0/N,beta1/N))

    
    Y=np.zeros((17))
    V = INP    
    
    infect_0=(V[1]+V[4]+V[7]+V[10]+V[13])
    infect_1=(V[6]+V[7]+V[8])
    
    
    Y[0]=-beta1*V[0]*infect_1-beta0*V[0]*infect_0+delta0*V[2]

    Y[1]=-beta1*V[1]*infect_1+beta0*V[0]*infect_0-gamma0*V[1]

    Y[2]=-beta1*V[2]*infect_1+gamma0*V[1]-delta0*V[2]



    Y[3]=beta1*V[0]*infect_1-sigma*V[3]-beta0*V[3]*infect_0+delta0*V[5]

    Y[4]=beta1*V[1]*infect_1-sigma*V[4]+beta0*V[3]*infect_0-gamma0*V[4]

    Y[5]=beta1*V[2]*infect_1-sigma*V[5]+gamma0*V[4]-delta0*V[5]



    Y[6]=sigma*V[3]-mu[0]*V[6]-omega*Y[6]-beta0*V[6]*infect_0+delta0*V[8]

    Y[7]=sigma*V[4]-mu[1]*V[7]-omega*Y[7]+beta0*V[6]*infect_0-gamma0*V[7]

    Y[8]=sigma*V[5]-mu[2]*V[8]-omega*Y[8]+gamma0*V[7]-delta0*V[8]



    Y[9]=omega*Y[6]-gamma1*Y[9]-beta0*V[9]*infect_0+delta0*V[11]

    Y[10]=omega*Y[7]-gamma1*Y[10]-mu[3]*V[10]+beta0*V[9]*infect_0-gamma0*V[10]

    Y[11]=omega*Y[8]-gamma1*Y[11]+gamma0*V[10]-delta0*V[11]



    Y[12]=gamma1*Y[9]-beta0*V[12]*infect_0+delta0*V[14]

    Y[13]=gamma1*Y[10]+beta0*V[12]*infect_0-gamma0*V[13]

    Y[14]=gamma1*Y[11]+gamma0*V[13]-delta0*V[14]



    Y[15]=mu[0]*V[6]+mu[2]*V[8]
    Y[16]=mu[1]*V[7]+mu[3]*V[10]

    
    return Y   # For odeint


In [10]:
N=2618
#sigma=1/14
IS0=600
RS0=0
SE0=2
IE0=0
RE0=0
SI0=0
II0=0
RI0=0
SX0=0
IX0=0
RX0=0
SR0=0
IR0=0
RR0=0
SS0=N-IS0-SE0-RS0
INPUT=(SS0,IS0,RS0,SE0,IE0,RE0,SI0,II0,RI0,SX0,IX0,RX0,SR0,IR0,RR0,0,0)
ts_len=len(measles_timeseries)
def g(t, INP, ps):
    RES = spi.odeint(coinf_sirs_model,INP,t,args=(ps,))
    res_measles=RES[1:ts_len+1,15]-RES[0:ts_len,15]
    res_coinf=RES[1:ts_len+1,16]-RES[0:ts_len,16]
    
    x=np.array((res_measles,res_coinf))
    #x = odeint(f, x0, t, args=(ps,))
    return x

def residual(ps,INP, ts, data):
    #x0 = ps['x0'].value
    #model = g(ts, x0, ps)
    RES = spi.odeint(coinf_sirs_model,INP,ts,args=(ps,))
    res_measles=RES[1:ts_len+1,15]-RES[0:ts_len,15]
    res_coinf=RES[1:ts_len+1,16]-RES[0:ts_len,16]
    
    model=np.array((res_measles,res_coinf))
    return (model - data).ravel()

# a = 2.0
# b = 0.5
# true_params = [a, b]
# x0 = 10.0

TS=1.0
ND=float(ts_len+1)
t_start = 0.0; t_end = ND; t_inc = TS
t_range = np.arange(t_start, t_end+t_inc, t_inc)

measles_ts_smooth = savgol_filter(measles_timeseries, 41, 3)
coinf_ts_smooth = savgol_filter(coinf_timeseries, 41, 3)
#time_series=np.array((measles_ts_smooth,coinf_ts_smooth))
time_series=np.array((measles_timeseries,coinf_timeseries))
# t = np.linspace(0, 10, 10)
# data = g(t, x0, true_params)
# data += np.random.normal(size=data.shape)

# set parameters incluing bounds
params = Parameters()
params.add('beta0',value=1.2,min=0.01,max=5)
params.add('beta1',value=0.5,min=0.01,max=5)
params.add('gamma0',value=0.1,min=0.01,max=0.9)
params.add('gamma1',value=0.1,min=0.001,max=0.9)
params.add('c1',value=0.1,min=0.0001,max=0.9)
params.add('c2',value=0.1,min=0.0001,max=0.9)
params.add('c3',value=0.1,min=0.0001,max=0.9)
params.add('c4',value=0.1,min=0.0001,max=0.9)
params.add('sigma',value=0.1,min=1/12,max=1/4)
params.add('omega',value=0.1,min=1/12,max=1/4)
#params.add('omega1',value=0.1,min=1/20,max=1/2)
params.add('delta0',value=1.0,min=0.9,max=1)
#params.add('alpha1',value=0.1,min=0,max=1)

# params.add('x0', value=float(data[0]), min=0, max=100)
# params.add('a', value= 1.0, min=0, max=10)
# params.add('b', value= 1.0, min=0, max=10)

# fit model and find predicted values
result = minimize(residual, params, args=(INPUT,t_range, time_series), method='leastsq')
final = time_series + result.residual.reshape(time_series.shape)


# beta0,beta1,gamma0,gamma1,c=1.2,1.2,0.1,0.1,0.1
# true_params = np.array((beta0,beta1,gamma0,gamma1,c))
# data = g(t_range, INPUT, true_params)
# data += np.random.normal(size=data.shape)
# result = minimize(residual, params, args=(INPUT,t_range, data), method='leastsq')
# final = data + result.residual.reshape(data.shape)


# plot data and fitted curves
# plt.plot(t, data, 'o')
# plt.plot(t, final, '--', linewidth=2, c='blue');

# # display fitted statistics
# report_fit(result)

/home/susanna/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [12]:
%matplotlib qt
plt.figure(figsize=(15,10))
plt.plot(time_series[0],label="Measles")
plt.plot(time_series[1],label="Coinfection")
plt.plot(final[0],label="fit to measles")
plt.plot(final[1],label="fit to coinfection")
# measles_ts_smooth = savgol_filter(measles_timeseries, 41, 3)
# coinf_ts_smooth = savgol_filter(coinf_timeseries, 41, 3)
# plt.plot(measles_ts_smooth)
# plt.plot(coinf_ts_smooth)
# beta0,beta1,gamma0,gamma1,c=1.2,1.5,0.1,0.1,0.3
# true_params = np.array((beta0,beta1,gamma0,gamma1,c))
# data = g(t_range, INPUT, true_params)
#data += np.random.normal(size=data.shape)
# plt.plot(data[0],label="Measles")
# plt.plot(data[1],label="coinf")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'PyQt4'

In [20]:
result.params.pretty_print()

Name       Value      Min      Max   Stderr     Vary     Expr Brute_Step
beta0     0.1921     0.01        5     None     True     None     None
beta1          4     0.01        5     None     True     None     None
c1        0.1292   0.0001      0.9     None     True     None     None
c2      0.005506   0.0001      0.9     None     True     None     None
c3       0.01411   0.0001      0.9     None     True     None     None
c4        0.0725   0.0001      0.9     None     True     None     None
delta0         1      0.9        1     None     True     None     None
gamma0    0.1123     0.01      0.9     None     True     None     None
gamma1   0.02401    0.001      0.9     None     True     None     None
omega     0.1106  0.08333     0.25     None     True     None     None
sigma    0.09586  0.08333     0.25     None     True     None     None


In [ ]:
def coinf_sirs_model2(INP,t,ps):  
    try:
        beta0=ps['beta0'].value
        beta1=ps['beta1'].value
#     beta=np.hstack((ps['beta0'].value,ps['beta1'].value))
        gamma0=ps['gamma0'].value
        gamma1=ps['gamma1'].value
        c1=ps['c1'].value
        c2=ps['c2'].value
        c3=ps['c3'].value
        c4=ps['c4'].value
        sigma=ps['sigma'].value
        omega=ps['omega'].value
        delta0=ps['delta0'].value

    except:
        beta0,beta1,gamma0,gamma1,c1,c2,c3,c4,sigma,omega,delta0=ps
    N=2618
    
    mu=np.array([c1*omega,c2*omega,c3*omega,c4*gamma1])
    beta=np.hstack((beta0/N,beta1/N))

    
    Y=np.zeros((17))
    V = INP    
    
    infect_0=(V[1]+V[4]+V[7]+V[10]+V[13])
    infect_1=(V[6]+V[7]+V[8])
    
    #Susceptible to measles
    
    Y[0]=-beta1*V[0]*infect_1-beta0*V[0]*infect_0+delta0*V[2]

    Y[1]=-beta1*V[1]*infect_1+beta0*V[0]*infect_0-gamma0*V[1]

    Y[2]=-beta1*V[2]*infect_1+gamma0*V[1]-delta0*V[2]

    #Exposed to measles

    Y[3]=beta1*V[0]*infect_1-sigma*V[3]-beta0*V[3]*infect_0+delta0*V[5]

    Y[4]=beta1*V[1]*infect_1-sigma*V[4]+beta0*V[3]*infect_0-gamma0*V[4]

    Y[5]=beta1*V[2]*infect_1-sigma*V[5]+gamma0*V[4]-delta0*V[5]

    #Infected with measles

    Y[6]=sigma*V[3]-mu[0]*V[6]-omega*Y[6]-beta0*V[6]*infect_0+delta0*V[8]+sigma*V[5]

    Y[7]=sigma*V[4]-mu[1]*V[7]-omega*Y[7]+beta0*V[6]*infect_0-gamma0*V[7]

    #get rid of
    Y[8]=sigma*V[5]-mu[2]*V[8]-omega*Y[8]+gamma0*V[7]-delta0*V[8]

    #Immunosuppressed from measles

    Y[9]=omega*Y[6]-gamma1*Y[9]-beta0*V[9]*infect_0+delta0*V[11]

    Y[10]=omega*Y[7]-gamma1*Y[10]-mu[3]*V[10]+beta0*V[9]*infect_0-gamma0*V[10]

    #get rid of
    Y[11]=omega*Y[8]-gamma1*Y[11]+gamma0*V[10]-delta0*V[11]

    #Recovered from measles

    Y[12]=gamma1*Y[9]-beta0*V[12]*infect_0+delta0*V[14]

    Y[13]=gamma1*Y[10]+beta0*V[12]*infect_0-gamma0*V[13]

    Y[14]=gamma1*Y[11]+gamma0*V[13]-delta0*V[14]



    Y[15]=mu[0]*V[6]+mu[2]*V[8]
    Y[16]=mu[1]*V[7]+mu[3]*V[10]

    
    return Y   # For odeint


In [1]:
report_fit(result)

NameError: name 'report_fit' is not defined